In [12]:
import sys
sys.path.append('../')

import tictactics_cython as tact
from tictactics_cython import Board
import MC
from MC import MC_node,mc_sim
from tictactics_policy import tictactics_simulation_policy as sim_pol
import matplotlib
import numpy as np
import cProfile
from matplotlib import pyplot as plt
import time
from IPython import display
%matplotlib inline

board = np.zeros((9,9))
for x in [(0,0),(0,4),(2,4),(0,8),(2,8),(4,3),(4,5)]:
    board[x]=1
for o in [(6,0),(7,0),(5,4),(5,5),(6,4),(6,8),(7,8),]:
    board[o]=-1
big_board = np.array([[0,0,0],
                      [0,0,0],
                      [0,0,0]])

player = 1
game = Board(board=board,big_board = big_board,player = player)

def one_game(game):
    while not game.over:
        log_len = len(game.log)
        root = MC_node(game,1)
        if game.player == 0:
            mc_sim(root,game,branch = 5,max_time = 1,passes = 1000000000,eval_fun = None)
        elif game.player == 1:
            mc_sim(root,game,branch = 5,max_time = 1,passes = 1000000000,eval_fun = sim_pol)

            

        #argmax
        max_i = -1
        max_N = -1
        for i in xrange(root.n_children):
            if root.N[i] > max_N:
                max_N = root.N[i]
                max_i = i
        move = root.actions[max_i]
            
            
        for _ in xrange(len(game.log) - log_len):
            game.erase_move()
        
        
        game.update_move(move)
#         display.clear_output(wait=True)
#         time.sleep(1)
#         print max_N
#         tact.fancy_board(game)
#         plt.show()
    return game.result
    
def x_games(game,sims = 100):
    log_len = len(game.log)
    p1wins=0
    p2wins=0
    draws =0
    
    for i in xrange(sims):
        result = one_game(game)
        if result == 1:
            p1wins+=1
        elif result == -1:
            p2wins+=1
        else:
            draws +=1

        for _ in xrange(len(game.log) - log_len):
            game.erase_move()
        
    sys.stdout.write('\r {} - {} - {}'.format(p1wins,draws,p2wins))
    
    return (p1wins,draws,p2wins)
    

C:\Users\Todd\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [13]:
''' 1s/move None wins more often than sim_pol'''
x_games(game,100)

15755751847673828622
this node has 6 children
move:(6, 5), runs:   179111, eval:-1.0000, principal:65             
move:(8, 5), runs:   179111, eval:-1.0000, principal:85 86 65       
move:(7, 4), runs:      555, eval:-0.1910, principal:74 85 86 36 21 65 
move:(7, 3), runs:      430, eval:+0.0419, principal:73 52 87 75 36 21 
move:(8, 4), runs:      401, eval:+0.1172, principal:84 65 16 52 86 38 27 73 42 36 
move:(7, 5), runs:      391, eval:+0.1458, principal:75 36 21 65 16 42 38 27 74 52 86 

65 
 54 - 22 - 24

(54, 22, 24)

In [8]:
x_games(game,20)

15201554821495253490
this node has 1 children
move:(8, 1), runs:  3649999, eval:+0.0000, principal:81             

81 
 11 - 0 - 9

(11, 0, 9)